In [134]:
from numba import njit
import numpy as np
from numpy import ndarray
import time
import matplotlib.pyplot as plt

In [135]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
input_str = '<' + input_str + '>'
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
           'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', '\'', '>']
print(f'{letters=}')
# print(f'{input_str=}')

len(letters)=32
502
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', "'", '>']


In [136]:
# generate SDR

SDR_len = 128 # 128
sparsity = 4 # 4
max_overlap = 1 # 1

sparse_prob = sparsity / SDR_len

letter_SDRs = {}

for letter in letters:
    while True:
        rnd_SDR = np.random.choice([0, 1], SDR_len, True, [1 - sparse_prob, sparse_prob])

        if (rnd_SDR > 0).sum() != sparsity:
            continue
        
        is_over_max = False
        for SDR in letter_SDRs.values():
            if (SDR & rnd_SDR).sum() > max_overlap:
                is_over_max = True
                break
        
        if is_over_max:
            continue
            
        break
    
    # print(f'{letter}')
    letter_SDRs.setdefault(letter, rnd_SDR)

for letter, SDR in letter_SDRs.items():
    print(f'{letter} : {(SDR > 0).sum()} {SDR.tolist()}')

a : 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
b : 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
c : 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [137]:
# text to SDRs
input_SDRs = []

for inp in input_str:
    input_SDRs.append(letter_SDRs[inp])

print(f'{len(input_SDRs)=}')

# for SDR in input_SDRs:
#     print(f'{SDR.tolist()}')

len(input_SDRs)=502


In [138]:
# init start column
column_size = 32

start_SDR = letter_SDRs['<']

start_I = np.zeros((start_SDR.size, column_size))

active_start_SDR_idx = np.where(start_SDR == 1)

for as_idx in active_start_SDR_idx[0]:
    rnd_active = np.random.randint(column_size)

    start_I[as_idx][rnd_active] = 1

# for col in np.rot90(start_I):
#     print(f'{col.astype(int).tolist()}')

In [139]:
I = np.zeros((SDR_len, column_size))
O = np.zeros((SDR_len, column_size))

W = np.random.rand(SDR_len, column_size, SDR_len * column_size) * 1e-2

V = np.zeros((SDR_len, column_size))

prec = 1e-30

# train

epoch_cnt = 1

eps0 = 0.003

eps0_V = 0.09

@njit()
def predict_train_one(I: ndarray, O: ndarray, W: ndarray, V: ndarray, prec: float, eps0: float, eps0_V: float, SDR: ndarray):
    rsh_I = I.flatten()

    O = np.sum((W * rsh_I), axis=2)

    O /= np.max(np.abs(O)) + prec

    max_V = np.zeros(V.shape[0])
    for max_idx in range(max_V.size):
        max_V[max_idx] = np.max(V[max_idx])

    max_V_T = max_V.reshape(1, max_V.size).T

    nV = np.abs(V - max_V_T)

    max_nV = np.zeros(nV.shape[0])
    for max_idx in range(max_nV.size):
        max_nV[max_idx] = np.max(nV[max_idx])

    nV /= max_nV.reshape(1, max_nV.size).T + prec

    OV = O + nV * eps0_V

    OV *= SDR.reshape(1, SDR.size).T

    max_O = np.zeros(O.shape[0])
    for max_idx in range(max_O.size):
        max_O[max_idx] = np.max(O[max_idx])

    O *= SDR.reshape(1, SDR.size).T
    O *= (OV >= max_O.reshape(1, max_O.size).T)
    max_O_idx = np.where(O != 0)

    for mO_idx in range(max_O_idx[0].size):
        x = max_O_idx[0][mO_idx]
        y = max_O_idx[1][mO_idx]

        for w_idx in range(W.shape[2]):
            W[x, y, w_idx] += (rsh_I[w_idx] - W[x, y, w_idx] * O[x, y]) * eps0

    # winners
    for mO_idx in range(max_O_idx[0].size):
        x = max_O_idx[0][mO_idx]
        y = max_O_idx[1][mO_idx]
        
        V[x, y] += 1

    # recur
    O *= (O != 0)

    I = np.copy(O)

    return I

for epoch_idx in range(epoch_cnt):
    I = np.copy(start_I)

    for SDR in input_SDRs[1:]:
        start = time.perf_counter()

        predict_train_one(I, O, W, V, prec, eps0, eps0_V, SDR)
        
        end = time.perf_counter()
        print("Elapsed (with compilation) = {}s".format((end - start)))

# O.shape
print(np.mean(V, axis=1).tolist())
print(np.sum(V == 0))
print(np.sum((V > 0), axis=1).tolist())
# for col in np.rot90(V):
#     print(f'{col.astype(int).tolist()}')

Elapsed (with compilation) = 8.579895199974999s
Elapsed (with compilation) = 0.7183153000660241s
Elapsed (with compilation) = 0.717994499951601s
Elapsed (with compilation) = 0.7170732000377029s
Elapsed (with compilation) = 0.8819145000306889s
Elapsed (with compilation) = 0.7976270000217482s
Elapsed (with compilation) = 0.7278129999758676s
Elapsed (with compilation) = 0.7276738000800833s
Elapsed (with compilation) = 0.8062629000050947s
Elapsed (with compilation) = 0.7611768998904154s
Elapsed (with compilation) = 0.7386689999839291s
Elapsed (with compilation) = 0.7158754999982193s
Elapsed (with compilation) = 0.7773995000170544s
Elapsed (with compilation) = 0.7250053000170738s
Elapsed (with compilation) = 0.7453494999790564s
Elapsed (with compilation) = 0.7235550000332296s
Elapsed (with compilation) = 0.7424762999871746s
Elapsed (with compilation) = 0.7332301000133157s
Elapsed (with compilation) = 0.8269073000410572s
Elapsed (with compilation) = 0.7360584000125527s
Elapsed (with compilat

KeyboardInterrupt: 

In [ ]:
# test
error = []

I = np.copy(start_I)

for SDR in input_SDRs[1:]:
    rsh_I = I.reshape(SDR_len * column_size)

    O = np.dot(W, rsh_I)

    max_O = np.max(O, axis=1)
    argmax_O_idx = np.argsort(max_O)[::-1][:sparsity]

    SDR_O = np.zeros(O.shape[0])
    SDR_O[argmax_O_idx] = 1

    err = (SDR.astype(int) ^ SDR_O.astype(int)).sum()
    # print(f'{err=}')
    error.append(err)

    O *= SDR[np.newaxis].T
    max_O = np.max(O, axis=1)
    O[O < max_O[np.newaxis].T] = 0

    # recur
    O[O != 0] = 1

    I = np.copy(O)

error = np.array(error)
mean_percent = int(np.mean(error) / (2 * sparsity) * 100)
print(f'{np.mean(error)=} {error.max()=} {error.min()=} {mean_percent=}')

# O.shape
# for col in np.rot90(O):
#     print(f'{col.astype(int).tolist()}')

np.mean(error)=7.772455089820359 error.max()=8 error.min()=4 mean_percent=97
